In [50]:
from climada.util.api_client import Client
client = Client()
tc = client.get_hazard('tropical_cyclone', properties={'country_iso3alpha': 'MEX', 'climate_scenario':'historical'})



https://climada.ethz.ch/data-api/v1/dataset	climate_scenario=historical	country_iso3alpha=MEX	data_type=tropical_cyclone	limit=100000	name=None	status=active	version=None


In [65]:
st = client.get_hazard('storm_europe', properties={'country_iso3alpha': 'DEU'})

https://climada.ethz.ch/data-api/v1/dataset	country_iso3alpha=DEU	data_type=storm_europe	limit=100000	name=None	status=active	version=None


In [51]:
tc = tc.select(orig=True)

In [60]:
rf = client.get_hazard('river_flood', properties={'country_iso3alpha': 'MEX', 'climate_scenario':'historical'})



https://climada.ethz.ch/data-api/v1/dataset	climate_scenario=historical	country_iso3alpha=MEX	data_type=river_flood	limit=100000	name=None	status=active	version=None


In [38]:
wf = client.get_hazard('wildfire', properties={'country_iso3alpha': 'MEX', 'climate_scenario':'historical'})



https://climada.ethz.ch/data-api/v1/dataset	climate_scenario=historical	country_iso3alpha=MEX	data_type=wildfire	limit=100000	name=None	status=active	version=None


In [42]:
[datetime.fromordinal(date) for date in wf.date]

[datetime.datetime(2001, 1, 1, 0, 0),
 datetime.datetime(2002, 1, 1, 0, 0),
 datetime.datetime(2003, 1, 1, 0, 0),
 datetime.datetime(2004, 1, 1, 0, 0),
 datetime.datetime(2005, 1, 1, 0, 0),
 datetime.datetime(2006, 1, 1, 0, 0),
 datetime.datetime(2007, 1, 1, 0, 0),
 datetime.datetime(2008, 1, 1, 0, 0),
 datetime.datetime(2009, 1, 1, 0, 0),
 datetime.datetime(2010, 1, 1, 0, 0),
 datetime.datetime(2011, 1, 1, 0, 0),
 datetime.datetime(2012, 1, 1, 0, 0),
 datetime.datetime(2013, 1, 1, 0, 0),
 datetime.datetime(2014, 1, 1, 0, 0),
 datetime.datetime(2015, 1, 1, 0, 0),
 datetime.datetime(2016, 1, 1, 0, 0),
 datetime.datetime(2017, 1, 1, 0, 0),
 datetime.datetime(2018, 1, 1, 0, 0),
 datetime.datetime(2019, 1, 1, 0, 0),
 datetime.datetime(2020, 1, 1, 0, 0)]

In [61]:
rf = rf.select(event_names=[event for event in rf.event_name if "matsiro" in event and "wfdei" in event])

In [52]:
tc_1980_2000 = tc.select(date=("1980-01-01","2000-01-01"))

In [53]:
tc_2000_2020 = tc.select(date=("2000-01-01","2020-01-01"))

In [45]:
def yearly_intensity_score(hazard):
    mat = hazard.intensity
    years = [datetime.fromordinal(date).year for date in hazard.date]
    if len(np.unique(years)) == len(hazard.event_id):
        return mat
    mask =[np.ma.make_mask(years == year).astype(int)
           for year in np.unique(years)]
    mask_matrix = sp.sparse.csr_matrix(mask)
    sum_mat = mask_matrix.dot(mat)
    return sum_mat


def corr_btw_hazards(hazard_dict, temporal, spatial):
    """
    Parameters
    ----------
    hazard_dict : dict of hazards
    temporal : bool, rather to consider the temporal dimension
    spatial : bool, rather to consider the spatial dimension

    Returns
    -------
    corr_df : pd.DataFrame
    """
    intensity_score_dict = {hazard: yearly_intensity_score(hazard_dict[hazard]) for hazard in hazard_dict}
    if temporal is True and spatial is False:
        df = pd.DataFrame.from_dict({hazard: np.squeeze(np.asarray(intensity_score_dict[hazard].sum(axis=0))) for hazard in intensity_score_dict})
    if spatial is True and temporal is False:
        df = pd.DataFrame.from_dict({hazard: np.squeeze(np.asarray(intensity_score_dict[hazard].sum(axis=1))) for hazard in intensity_score_dict})
    if spatial is True and temporal is True:
        df = pd.DataFrame.from_dict({hazard: np.array(intensity_score_dict[hazard].todense().flatten())[0]
                                     for hazard in intensity_score_dict})
    
    return df.corr()


In [62]:
corr_btw_hazards({'TC':tc_1980_2000,'RF':rf}, temporal=True, spatial=False)

,TC,RF
TC,1.000000,0.039743
RF,0.039743,1.000000


In [64]:
corr_btw_hazards({'TC':tc_2000_2020,'WF':wf}, temporal=True, spatial=True)

,TC,WF
TC,1.000000,0.065908
WF,0.065908,1.000000
